In [31]:
%matplotlib inline
import pandas
import matplotlib
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

columns = ['hour', 'last_windspd', 'last_no', 'last_no2', 'last_nox', 'last_o3', 'no', 'no2', 'nox', 'o3']

plt.rcParams["figure.figsize"] = (20, 12)
plt.rcParams.update({'font.size': 22})

df = pandas.read_csv('data00_100000_clean.csv')
df = df.drop(['AQS_Code', 'Latitude', 'Longitude', 'epoch', 'winddir', 'time', 'date', 'day'], axis=1)
df['last_windspd'] = df['windspd'].shift(1)
df['last_no'] = df['no'].shift(1)
df['last_no2'] = df['no2'].shift(1)
df['last_nox'] = df['nox'].shift(1)
df['last_o3'] = df['o3'].shift(1)
df = df.drop(df.index[[0]])

scaler = MinMaxScaler()
scaler.fit(df[columns])

df = scaler.transform(df[columns])

# Create sequences
sequence_length = 32
num_inputs = 6
num_outputs = 4


data = []
labels = []

rows = deque(maxlen=sequence_length)

for idx, r in enumerate(df):

    rows.append([r[0], r[1], r[2], r[3], r[4], r[5]])
    
    if len(rows) < sequence_length:
        continue
        
    data.append(rows.copy())
    
    labels.append([r[6], r[7], r[8], r[9]])

data = np.array(data)
labels = np.array(labels)

/Users/carroll/anaconda3/envs/pollen/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [32]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# custom R2-score metrics for keras backend
from keras import backend as K

def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

model = Sequential()
model.add(Dense(num_inputs, input_shape=(sequence_length,num_inputs)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_outputs))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[r2_keras])
model.summary()

model.fit(x=data, y=labels, batch_size=128, epochs=20, validation_split=0.33, verbose=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 32, 6)             42        
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                18176     
_________________________________________________________________
dense_17 (Dense)             (None, 256)               16640     
_________________________________________________________________
dense_18 (Dense)             (None, 4)                 1028      
Total params: 35,886
Trainable params: 35,886
Non-trainable params: 0
_________________________________________________________________
Train on 56538 samples, validate on 27848 samples
Epoch 1/20
56538/56538 [==============================] - 64s 1ms/step - loss: 3.6136e-04 - r2_keras: 0.9360 - val_loss: 1.9303e-04 - val_r2_keras: 0.9499
Epoch 2/20
56538/56538 [==============================] - 93s 2ms

KeyboardInterrupt: 